In [ ]:
from multiprocessing import Process
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

import cv2
import netCDF4 as nc
import matplotlib.pyplot as plt

from subprocess import run, DEVNULL

from torch.nn import Sequential, Sigmoid
from torch import from_numpy, inference_mode
from torch.nn import DataParallel

from monai.networks.nets import SwinUNETR
from monai.inferers import sliding_window_inference

from pytorch_lightning import LightningModule

from zipfile import ZipFile
from zipfile import BadZipFile

import xmltodict

import os
from os.path import join, exists, getsize, isfile

In [ ]:
class DownloadSARSentinel:
    def __init__(self, email, 
             password,
             product_list,
             satellite="sentinel_1"
                ):
        self.password = password
        self.email = email
        self.product_list = product_list
        self.satellite = satellite
        self.raw_folder = None
        self._query_result = None

    def query_result(self):
        if not isinstance(self._query_result, pd.DataFrame):
            self._query_result = self.query_sar()
        return self._query_result
        
    def query_sar(self):
        API_URL_NAME = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'{name}')"
        query_result = pd.DataFrame()

        for product in self.product_list:
            json = requests.get(API_URL_NAME.format(name=product)).json()
            response_result = pd.DataFrame.from_dict(json['value'])

            if not response_result.empty:
                print("Found:", product)
                query_result = pd.concat([query_result, response_result])   
                #yield response_result
            else:
                print("Could not find: ", product)

        return query_result
        
    @staticmethod
    def create_folder(folder_path, original_name):
        if not folder_path:
            folder_path = join(os.getcwd(), original_name)
            if not exists(folder_path):
                os.mkdir(folder_path)

        assert exists(folder_path), f"Path not found: {folder_path}"
        return folder_path

    @staticmethod
    def is_downloaded(sar_name, folder):
        sar_path = join(folder, sar_name + ".zip")
        if exists(sar_path):
            #if getsize(sar_path) > 5E9: #ver o tamanho do arquivo para baixar arquivos incompletos
            return True
        return False

    def return_headers(self):
        token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        data = {
            'grant_type': 'password',
            'username': self.email,
            'password': self.password,
            'client_id': 'cdse-public'
        }
        token_response = requests.post(token_url, headers=headers, data=data).json()
        token_url = token_response["access_token"]
        download_headers = {"Authorization": f"Bearer {token_url}"}
        return download_headers

    def download_products(self, folder, overwrite=False):
        download_url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"

        for index, product in self.query_result().iterrows():
            path_product = join(folder, product["Name"] + ".zip")               
            if not DownloadSARSentinel.is_downloaded(product["Name"], folder) or overwrite:
                session = requests.Session()
                session.headers.update(self.return_headers())
                response = session.get(download_url.format(product_id=product["Id"]), headers=self.return_headers(), stream=True)
                total_size = int(response.headers.get('Content-Length', 0))
                progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, desc=f'Downloading: {product["Name"]}', leave=True)
    
                with open(path_product, "wb") as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            file.write(chunk)
                            progress_bar.update(len(chunk))
                progress_bar.reset()




        

In [ ]:
QUERY_CSV_PATH = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/New_sar_img.csv"
FILE_NAME_COLUMN = "sar_file_name"
query_sar = pd.read_csv(QUERY_CSV_PATH, header=0)[FILE_NAME_COLUMN]
query_sar = query_sar[2:4]
#print(query_sar)

test = DownloadSARSentinel("pedro.meirelles@ufba.br", "Thermal1234@", product_list=list(query_sar))

test.query_result()
test.download_products(folder="/mnt/camobi_2/PHMG/Sentinel_Acquisition/raw_folder")

In [ ]:
class SentinelProduct:
    def __init__(self, name, file_path, nc_graph_path,  rect_corners=None):
        self.name = name
        self.rect_corners = rect_corners
        self.file_path = file_path
        self.nc_graph_path = nc_graph_path
        self.unzip_path = None
        self.netcdf_path = None

    def unzip(self, unzip_folder):
        assert exists(unzip_folder), f"Folder \"{unzip_folder}\" does not exist!"
        self.unzip_path = join(unzip_folder, self.name) 

        try:
            print("Unziping:", self.name)
            with ZipFile(self.file_path, 'r') as zip_ref:
                zip_ref.extractall(self.unzip_path)
        except BadZipFile:
            print(f"Imposible to unzip: {self.name}. file is incomplete or corrupted!")

    def edit_zip_to_nc(self, netcdf_folder):
        with open(self.nc_graph_path) as arquivo:
            dados = xmltodict.parse(arquivo.read())

        if not self.unzip_path:
            input_path = self.unzip_path
        else:
            input_path = self.file_path

        self.netcdf_path = join(netcdf_folder, self.name + ".nc")

        dados['graph']['node'][0]['parameters']['file'] = input_path 
        dados['graph']['node'][-1]['parameters']['file'] = self.netcdf_path 

        with open(self.nc_graph_path, 'w') as arquivo:
            arquivo.write(xmltodict.unparse(dados, pretty=True))

    def convert_to_netcdf4(self, gpt_path, netcdf_folder):
        self.edit_zip_to_nc(netcdf_folder)
        shell = run([gpt_path, self.nc_graph_path])#, stdout=DEVNULL, stderr=DEVNULL)



In [ ]:
SAR_TO_NC_GRAPH = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/graphs/ZIP_to_NC.xml"
PATH_TO_GPT = "/home/camobi/snap/bin/gpt"
NETCDF_FOLER = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/netcdf_folder"
UNZIP_FOLDER = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/unzip_folder"


product_name = pd.read_csv(QUERY_CSV_PATH, header=0)[FILE_NAME_COLUMN][0][:-5]
product_path = join("/mnt/camobi_2/PHMG/Sentinel_Acquisition/raw_folder", product_name + ".SAFE.zip")

product = SentinelProduct(product_name, product_path, SAR_TO_NC_GRAPH)
product.unzip(UNZIP_FOLDER)
product.convert_to_netcdf4(PATH_TO_GPT, NETCDF_FOLER)

In [ ]:
class PedroNet(LightningModule): #out_channels = numero de classes
    def __init__(self, img_size, lr,
                 depths=(2, 2, 2, 2), 
                 num_heads=(3, 6, 12, 24), 
                 feature_size=24, 
                 norm_name='instance', 
                 drop_rate=0.0, 
                 attn_drop_rate=0.0, 
                 dropout_path_rate=0.0, 
                 normalize=True, 
                 use_checkpoint=False, 
                 downsample='merging', 
                 use_v2=False 
                 ):
        super().__init__()
        self.model = Sequential(SwinUNETR(spatial_dims=2,
                                    in_channels=1,
                                    out_channels=1,
                                    depths=depths,
                                    img_size=img_size,
                                    feature_size=feature_size,
                                    drop_rate=drop_rate,
                                    num_heads=num_heads,
                                    norm_name=norm_name,
                                    attn_drop_rate=attn_drop_rate,
                                    dropout_path_rate=dropout_path_rate,
                                    normalize=normalize,
                                    use_checkpoint=use_checkpoint,
                                    downsample=downsample,
                                    #use_v2=use_v2, apenas para versões mais recentes
                                    ))
    def forward(self, x):
        return self.model(x)
    
    def netcdf_inferece(self, nc_product):
        torch_img = from_numpy(nc_product.nc_img).to("cuda")
        torch_img = torch_img.unsqueeze(0).unsqueeze(0)

        with inference_mode():
            model_img = sliding_window_inference(torch_img, 
                                            roi_size=(512),
                                            sw_batch_size=20, 
                                            predictor=DataParallel(self.model), 
                                            mode='gaussian',
                                            overlap=0.3,
                                            progress=True
                                            )

            sigmoid_fn = Sigmoid()
            model_img = sigmoid_fn(model_img)
            model_img = model_img.to("cpu")
        return model_img

class NetcdfProduct:
    def __init__(self, product, image_variable='Sigma0_VV_db'):
        self.product = product
        self.name = product.filepath().split("/")[-1].split(".")[0]
        self.image_variable = image_variable
        self.nc_img = np.asarray(product.variables[self.image_variable][:])

    def create_img(self, folder_path):
        img_path = join(folder_path, self.name + ".png")
        normalized_img = ((self.nc_img - np.min(self.nc_img)) / (np.max(self.nc_img) - np.min(self.nc_img))) * 255
        cv2.imwrite(img_path, normalized_img)
    
    @staticmethod
    def create_polygons(mask):
        edited_contours = []
        binary_image = np.array(mask)
        
        contours, hierarchy = cv2.findContours(binary_image.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for poly in contours:
            if len(poly) > 50:
                approx = cv2.approxPolyDP(poly, 0.8, True)
                approx = np.squeeze(approx)
                edited_contours.append(approx)
        return edited_contours

    def polygon_to_labelme(self, polygons, mask):
        masked_polygons = NetcdfProduct.create_polygons(mask)
        labelme_format = {"version": "5.1.1",
                          "flags": {},
                          "shapes": [],
                          "imagePath": f"..\\Sar_img\\{self.name}.png",  # Update with your image filename
                          "imageData": None,
                          "imageHeight": self.product.dimensions["y"],
                          "imageWidth": self.product.dimensions["x"]
                          }

        for patch in polygons:
            labelme_format["shapes"].append({
                "label": "oil", 
                "points": patch.squeeze().tolist(),
                "group_id": None,
                "description": "",
                "shape_type": "polygon",
                "flags": {}
            })


In [ ]:
NETCDF_PRODUCT_PATH = "/mnt/camobi_process/new_data/images_nc/4A2D.nc"

nc_img = nc.Dataset(NETCDF_PRODUCT_PATH, 'r')
netcdf_sar = NetcdfProduct(nc_img)
netcdf_sar.product
#netcdf_sar.create_img("/mnt/camobi_2/PHMG/Sentinel_Acquisition/img_folder")

In [ ]:
WEIGHTS_MODEL = "/mnt/camobi_2/PHMG/PedroSwinNet/Model_512Img_24Feature_(2, 2, 2, 2)depths_0.0attnDrop_(3, 6, 12, 24)Heads_30.000000Lr_0drop_v1/model-Val_loss=0.003714-Val_Precision=0.973-Recall=0.971-Val_F1_Score=0.972.ckpt"
loaded_model = PedroNet.load_from_checkpoint(WEIGHTS_MODEL)
loaded_model.netcdf_inferece(netcdf_sar)